In [1]:
import pandas as pd
import pickle
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
loyalty_scores = pd.read_excel("grocery_database.xlsx", sheet_name="loyalty_scores")
customer_details = pd.read_excel("grocery_database.xlsx", sheet_name="customer_details")
transactions = pd.read_excel("grocery_database.xlsx", sheet_name = "transactions")


In [3]:
loyalty_scores

,customer_id,customer_loyalty_score
0,104,0.587
1,69,0.156
2,525,0.959
3,181,0.418
4,796,0.570
...,...,...
395,705,0.637
396,864,0.657
397,346,0.410
398,511,0.687


In [4]:
customer_details

,customer_id,distance_from_store,gender,credit_score
0,74,3.38,F,0.59
1,524,4.76,F,0.52
2,607,4.45,F,0.49
3,343,0.91,M,0.54
4,322,3.02,F,0.63
...,...,...,...,...
865,372,4.38,F,0.50
866,104,2.36,F,0.63
867,393,1.87,M,0.59
868,373,0.21,M,0.47


In [5]:
transactions

,customer_id,transaction_date,transaction_id,product_area_id,num_items,sales_cost
0,1,2020-04-10,435657533999,3,7,19.16
1,1,2020-04-10,435657533999,2,5,7.71
2,1,2020-06-02,436189770685,4,4,26.97
3,1,2020-06-02,436189770685,1,2,38.52
4,1,2020-06-10,436265380298,4,4,22.13
...,...,...,...,...,...,...
38501,870,2020-09-23,437316479610,5,4,51.61
38502,870,2020-09-23,437316479610,2,15,33.77
38503,870,2020-09-30,437385547544,5,3,23.46
38504,870,2020-09-30,437385547544,1,1,18.52


In [6]:
#Create the customer level data set

data_for_regression = pd.merge(customer_details, loyalty_scores, how="left", on="customer_id")

In [7]:
data_for_regression

,customer_id,distance_from_store,gender,credit_score,customer_loyalty_score
0,74,3.38,F,0.59,0.263
1,524,4.76,F,0.52,0.298
2,607,4.45,F,0.49,0.337
3,343,0.91,M,0.54,0.873
4,322,3.02,F,0.63,0.350
...,...,...,...,...,...
865,372,4.38,F,0.50,0.321
866,104,2.36,F,0.63,0.587
867,393,1.87,M,0.59,NaN
868,373,0.21,M,0.47,0.972


In [8]:
#Aggregate sales data here
#Create 5 sales metrics

sales_summary = transactions.groupby("customer_id").agg({"sales_cost" : "sum",
                                                        "num_items": "sum",
                                                        "transaction_id" : "count",
                                                        "product_area_id" : "nunique"}).reset_index()

In [9]:
sales_summary
#These are summaries across each customer id here

,customer_id,sales_cost,num_items,transaction_id,product_area_id
0,1,3980.49,424,51,5
1,2,2056.91,213,52,5
2,3,324.22,65,12,4
3,4,3499.39,278,47,5
4,5,6609.19,987,106,5
...,...,...,...,...,...
865,866,1005.04,226,49,4
866,867,2249.73,244,52,5
867,868,1748.45,329,56,5
868,869,1209.85,184,54,5


In [10]:
sales_summary.columns = ["customer_id", "total_sales", "total_items", "transaction_count", "product_area_count"]

In [11]:
#Avg amount customers spend during each visit
sales_summary["average_basket_value"] = sales_summary["total_sales"]/sales_summary["transaction_count"]

In [12]:
sales_summary

,customer_id,total_sales,total_items,transaction_count,product_area_count,average_basket_value
0,1,3980.49,424,51,5,78.048824
1,2,2056.91,213,52,5,39.555962
2,3,324.22,65,12,4,27.018333
3,4,3499.39,278,47,5,74.455106
4,5,6609.19,987,106,5,62.350849
...,...,...,...,...,...,...
865,866,1005.04,226,49,4,20.511020
866,867,2249.73,244,52,5,43.264038
867,868,1748.45,329,56,5,31.222321
868,869,1209.85,184,54,5,22.404630


In [13]:
data_for_regression = pd.merge(data_for_regression, sales_summary, how="inner", on="customer_id")

In [14]:
data_for_regression

,customer_id,distance_from_store,gender,credit_score,customer_loyalty_score,total_sales,total_items,transaction_count,product_area_count,average_basket_value
0,74,3.38,F,0.59,0.263,2563.71,297,44,5,58.266136
1,524,4.76,F,0.52,0.298,2996.02,357,49,5,61.143265
2,607,4.45,F,0.49,0.337,2853.82,350,49,5,58.241224
3,343,0.91,M,0.54,0.873,2388.31,272,54,5,44.227963
4,322,3.02,F,0.63,0.350,2401.64,278,50,5,48.032800
...,...,...,...,...,...,...,...,...,...,...
865,372,4.38,F,0.50,0.321,3423.01,338,49,5,69.857347
866,104,2.36,F,0.63,0.587,3648.08,280,49,5,74.450612
867,393,1.87,M,0.59,NaN,3067.83,254,53,5,57.883585
868,373,0.21,M,0.47,0.972,3303.80,264,51,5,64.780392


In [15]:
#Split for those with loyalty scores, and those without

#Where loyalty score is availabe

regression_modelling = data_for_regression.loc[data_for_regression["customer_loyalty_score"].notna()]
regression_scoring = data_for_regression.loc[data_for_regression["customer_loyalty_score"].isna()]

In [16]:
regression_modelling

,customer_id,distance_from_store,gender,credit_score,customer_loyalty_score,total_sales,total_items,transaction_count,product_area_count,average_basket_value
0,74,3.38,F,0.59,0.263,2563.71,297,44,5,58.266136
1,524,4.76,F,0.52,0.298,2996.02,357,49,5,61.143265
2,607,4.45,F,0.49,0.337,2853.82,350,49,5,58.241224
3,343,0.91,M,0.54,0.873,2388.31,272,54,5,44.227963
4,322,3.02,F,0.63,0.350,2401.64,278,50,5,48.032800
...,...,...,...,...,...,...,...,...,...,...
862,4,2.14,M,0.64,0.564,3499.39,278,47,5,74.455106
865,372,4.38,F,0.50,0.321,3423.01,338,49,5,69.857347
866,104,2.36,F,0.63,0.587,3648.08,280,49,5,74.450612
868,373,0.21,M,0.47,0.972,3303.80,264,51,5,64.780392


In [17]:
regression_scoring

,customer_id,distance_from_store,gender,credit_score,customer_loyalty_score,total_sales,total_items,transaction_count,product_area_count,average_basket_value
6,1,4.78,F,0.66,NaN,3980.49,424,51,5,78.048824
7,120,3.49,F,0.38,NaN,2887.20,253,45,5,64.160000
8,52,14.91,F,0.68,NaN,3342.75,335,47,5,71.122340
10,435,0.25,M,0.62,NaN,2326.71,267,48,5,48.473125
12,679,4.74,F,0.58,NaN,3448.59,370,49,5,70.379388
...,...,...,...,...,...,...,...,...,...,...
860,711,400.97,F,0.54,NaN,4072.86,346,45,5,90.508000
861,309,0.32,M,0.38,NaN,3097.99,281,51,5,60.744902
863,765,1.62,M,0.63,NaN,3736.02,250,46,5,81.217826
864,466,4.36,F,0.54,NaN,4407.12,354,47,5,93.768511


In [18]:
regression_scoring.drop(["customer_loyalty_score"], axis=1, inplace=True)

C:\Users\19144\AppData\Local\Temp\ipykernel_2492\1823117083.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regression_scoring.drop(["customer_loyalty_score"], axis=1, inplace=True)


In [19]:
pickle.dump(regression_modelling, open(r"C:\Users\19144\OneDrive\Desktop\Data Science Infinity\abc_regression_modeling.p", "wb"))
pickle.dump(regression_scoring, open(r"C:\Users\19144\OneDrive\Desktop\Data Science Infinity\abc_regression_scoring.p", "wb"))


In [20]:
import pickle

# Assuming regression_modelling and regression_scoring are defined somewhere in your code

# Save regression_modelling object
with open(r"C:\Users\19144\OneDrive\Desktop\Data Science Infinity\Python Fundamentals\abc_regression_modeling.p", "wb") as f:
    pickle.dump(regression_modelling, f)

# Save regression_scoring object
with open(r"C:\Users\19144\OneDrive\Desktop\Data Science Infinity\Python Fundamentals\ abc_regression_scoring.p", "wb") as f:
    pickle.dump(regression_scoring, f)
